# AnnoSeqCell class

> Read table and add annotation

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

In [ ]:
#| default_exp annoseqcell

In [ ]:
#| export
import numpy as np
import pandas as pd
import gzip
from digestion_rates.preprocess import SequencesCells
from pathlib import Path
from multiprocessing import Pool

# AnnoSeqCell class





In [ ]:
#| export
class AnnoSequencesCells(SequencesCells):
    def __init__(self, df=None):
        super().__init__(df)
        self.var = None
    def add_cell_anno(self, path, compression='gzip'):
        anno_df = pd.read_csv(path, compression=compression)
        anno_df.columns = map(lambda x: x.lower(), anno_df.columns)
        self.var = anno_df
        
    

In [ ]:
df = pd.read_csv('../RPMD1/meta_data/cell_annotations.csv.gz')
df.columns = map(lambda x: x.lower(), df.columns)
df

,cb,cds_frac,units_mnase,sort_population
0,RPMD1-03_AACAGCAATG,0.824841,25.000000,Interphase
1,RPMD1-03_AACCACGTCA,0.899036,0.390625,Interphase
2,RPMD1-03_AACCGTAACA,0.786022,0.048828,Interphase
3,RPMD1-03_AACCTAGACG,0.400000,0.000000,Interphase
4,RPMD1-03_AACGCGGTAG,0.500000,0.000000,Interphase
...,...,...,...,...
6126,RPMD1-18_TTGGTGTGTC,0.780000,0.048828,G0
6127,RPMD1-18_TTGTCTCTAC,0.716981,0.012207,Mit
6128,RPMD1-18_TTGTGCTTGG,1.000000,0.000000,Mit
6129,RPMD1-18_TTGTTCGTGT,0.915308,25.000000,Interphase


In [ ]:
data = [('ACCA', 'PLATE-01_CCCC', 1), ('ACCA', 'PLATE-01_CCCA', 1), ('CAAC', 'PLATE-01_CCCA', 2), ('ACCA', 'PLATE-02_CCCC', 1), ('CAAC', 'PLATE-02_CCCT', 2)]
df = pd.DataFrame(data, columns=['seq', 'cb', 'counts'])
SequencesCells(df).to_wide_format()

AttributeError: 'Series' object has no attribute 'plate'